In [3]:
pip install tensorboardX

     |████████████████████████████████| 124 kB 8.4 MB/s 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<h3><center>Imports</center></h3>

In [4]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import os
import scipy
import numpy as np
from skimage.measure import label, regionprops
import cv2
import math
import torch
import random
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms import functional as TF
from skimage.transform import rotate, resize
import argparse
from tqdm import tqdm
import torch.optim as optim
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from tensorboardX import SummaryWriter
from torch.utils.data import DataLoader

<h3><center>RFCN Model</center></h3>

In [5]:
__all__ = ['RFCN']

def conv3x3(in_channels, out_channels, stride=1):
         return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, recurrent_n=2, active_func=nn.ReLU(inplace=True)):
        super(BasicBlock, self).__init__()
        self.block = nn.Sequential(
            conv3x3(in_planes, planes),
            nn.BatchNorm2d(planes),
            active_func,
            conv3x3(planes, planes),
            nn.BatchNorm2d(planes),
            active_func
        )

    def forward(self, x):
        return self.block(x)


class RecurrentBlock(nn.Module):
    def __init__(self, in_planes, planes, recurrent_n=2, active_func=nn.ReLU(inplace=True)):
        super(RecurrentBlock, self).__init__()
        self.recurrent_n = recurrent_n
        self.conv1 = conv3x3(in_planes, planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.active_func = active_func

        self.RCL = conv3x3(planes, planes)
        self.bn2 = nn.ModuleList()
        for n in range(recurrent_n):
            self.bn2.append(nn.BatchNorm2d(planes))

    def forward(self, x):
        conv1 = self.bn1(self.conv1(x))
        active = self.active_func(conv1)
        for n in range(self.recurrent_n):
            conv2 = self.bn2[n](self.RCL(active))
            active = self.active_func(conv1 + conv2)

        return active


class RecurrentBlock2(nn.Module):
    def __init__(self, in_planes, planes, recurrent_n=2, active_func=nn.ReLU(inplace=True)):
        super(RecurrentBlock2, self).__init__()
        self.recurrent_basic_block1 = RecurrentBlock(in_planes, planes, recurrent_n=recurrent_n, active_func=active_func)
        self.recurrent_basic_block2 = RecurrentBlock(planes, planes, recurrent_n=recurrent_n, active_func=active_func)

    def forward(self, x):
        out = self.recurrent_basic_block1(x)
        out = self.recurrent_basic_block2(out)
        return out


class RecurrentBasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1, recurrent_n=2, active_func=nn.ReLU(inplace=True)):
        super(RecurrentBasicBlock, self).__init__()
        self.recurrent_n = recurrent_n
        self.recurrent_basic_block_1 = BasicBlock(in_planes, planes, recurrent_n=recurrent_n, active_func=active_func)
        self.recurrent_basic_block_2 = BasicBlock(planes, planes, recurrent_n=recurrent_n, active_func=active_func)

    def forward(self, x):
        out = self.recurrent_basic_block_1(x)

        out_2 = out
        for n in range(self.recurrent_n):
            out_2 = self.recurrent_basic_block_2(out_2) + out

        return out_2


class RecurrentBasicBlock2(nn.Module):
    def __init__(self, in_planes, planes, stride=1, recurrent_n=2, active_func=nn.ReLU(inplace=True)):
        super(RecurrentBasicBlock2, self).__init__()
        self.recurrent_n = recurrent_n
        self.recurrent_basic_block_1 = RecurrentBasicBlock(in_planes, planes, recurrent_n=recurrent_n, active_func=active_func)
        self.recurrent_basic_block_2 = RecurrentBasicBlock(planes, planes, recurrent_n=recurrent_n, active_func=active_func)

    def forward(self, x):
        out = self.recurrent_basic_block_1(x)
        out = self.recurrent_basic_block_2(out)
        return out


class BasicNet(nn.Module):

    def __init__(self, filters, block, recurrent_n=2, layers=4, kernel=3, inplanes=3, num_classes=2, active_func=None):
        super().__init__()
        self.filters = filters
        self.layers = layers
        self.kernel = kernel
        self.inplanes = inplanes
        self.active_func = active_func

        self.leakyRelu = nn.LeakyReLU(negative_slope=0.2, inplace=True)
        self.relu = nn.ReLU(inplace=True)

        self.down_conv_list = nn.ModuleList()
        self.down_conv_2_list = nn.ModuleList()
        self.down_scale_conv_list = nn.ModuleList()
        for i in range(layers):
            self.down_conv_list.append(block(inplanes if i == 0 else filters * 2 ** i, filters * 2 ** i,
                                             recurrent_n=recurrent_n, active_func=self.leakyRelu))
            self.down_conv_2_list.append(nn.Sequential(
                conv3x3(filters * 2 ** i, filters * 2 ** (i + 1), 2),
                nn.BatchNorm2d(filters * 2 ** (i + 1))
            ))
            if i != layers - 1:
                self.down_scale_conv_list.append(nn.Sequential(
                    conv3x3(inplanes, filters * 2 ** (i + 1)),
                    nn.BatchNorm2d(filters * 2 ** (i + 1))
                ))

        self.bottom = block(filters * 2 ** layers, filters * 2 ** layers, recurrent_n=recurrent_n, active_func=self.leakyRelu)

        self.up_convtrans_list = nn.ModuleList()
        self.up_conv_list = nn.ModuleList()
        self.out_conv = nn.ModuleList()
        for i in range(0, layers):
            self.up_convtrans_list.append(nn.Sequential(
                nn.ConvTranspose2d(in_channels=filters * 2 ** (layers - i), out_channels=filters * 2 ** max(0, layers - i - 1),
                                   kernel_size=3, stride=2, padding=1, output_padding=1, bias=True),
                nn.BatchNorm2d(filters * 2 ** max(0, layers - i - 1))
            ))
            self.up_conv_list.append(block(filters * 2 ** max(0, layers - i - 1), filters * 2 ** max(0, layers - i - 1),
                                           recurrent_n=recurrent_n, active_func=self.relu))

            self.out_conv.append(conv3x3(filters * 2 ** (self.layers - i - 1), num_classes))

    def forward(self, x):
        x_down = down_out = x
        down_outs = []
        for i in range(0, self.layers):
            down_out = self.down_conv_list[i](down_out)
            down_outs.append(down_out)
            down_out = self.down_conv_2_list[i](down_out)
            if i != self.layers - 1:
                x_down = F.avg_pool2d(x_down, 2)
                down_out = self.leakyRelu(down_out + self.down_scale_conv_list[i](x_down))
            else:
                down_out = self.leakyRelu(down_out)

        # bottom branch
        up_out = self.bottom(down_out)

        outs = []
        out_final = None
        for j in range(self.layers):
            up_out = self.relu(down_outs[self.layers - j - 1] + self.up_convtrans_list[j](up_out))
            up_out = self.up_conv_list[j](up_out)

            out = F.interpolate(up_out, size=(up_out.shape[2] * 2 ** (self.layers - j - 1), up_out.shape[3] * 2 ** (self.layers - j - 1)))
            out = self.out_conv[j](out)
            if self.active_func is not None:
                out = self.active_func(out, dim=1)
            outs.append(out)
            out_final = out if out_final is None else out_final + out

        out_final /= self.layers
        outs.append(out_final)
        return outs


def RFCN(filters, layers=4, kernel=3, inplanes=3, num_classes=2, active_func=None, recurrent_n=2):
    return BasicNet(filters=filters, block=RecurrentBasicBlock2, layers=layers, kernel=kernel, inplanes=inplanes, num_classes=num_classes,
                    active_func=active_func, recurrent_n=recurrent_n)

<h3><center>Utils Functions</center></h3>

In [6]:
def mkdir_p(path):
    try:
        os.makedirs(path)
        return path
    except OSError as exc:  # Python >2.5
        import errno
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise


def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']


def BW_img(input, thresholding):
    if input.max() > thresholding:
        binary = input > thresholding
    else:
        binary = input > input.max() / 2.0
    label_image = label(binary)
    regions = regionprops(label_image)
    area_list = []
    for region in regions:
        area_list.append(region.area)
    if area_list:
        idx_max = np.argmax(area_list)
        binary[label_image != idx_max + 1] = 0
    return scipy.ndimage.binary_fill_holes(np.asarray(binary).astype(int))


def disc_crop(org_img, DiscROI_size, C_x, C_y):
    tmp_size = int(DiscROI_size / 2)
    disc_region = np.zeros((DiscROI_size, DiscROI_size, 3), dtype=org_img.dtype)  # (400, 400, 3)
    crop_coord = np.array([C_x - tmp_size, C_x + tmp_size, C_y - tmp_size, C_y + tmp_size], dtype=int)  # [787, 1187, 146, 546]
    err_coord = [0, DiscROI_size, 0, DiscROI_size]  # [0, 400, 0, 400]

    if crop_coord[0] < 0:
        err_coord[0] = abs(crop_coord[0])
        crop_coord[0] = 0

    if crop_coord[2] < 0:
        err_coord[2] = abs(crop_coord[2])
        crop_coord[2] = 0

    if crop_coord[1] > org_img.shape[0]:
        err_coord[1] = err_coord[1] - (crop_coord[1] - org_img.shape[0])
        crop_coord[1] = org_img.shape[0]

    if crop_coord[3] > org_img.shape[1]:
        err_coord[3] = err_coord[3] - (crop_coord[3] - org_img.shape[1])
        crop_coord[3] = org_img.shape[1]

    disc_region[err_coord[0]:err_coord[1], err_coord[2]:err_coord[3], ] = org_img[crop_coord[0]:crop_coord[1], crop_coord[2]:crop_coord[3], ]

    return disc_region, err_coord, crop_coord


def return_list(data_path, data_type):
    file_list = [file for file in os.listdir(data_path) if file.lower().endswith(data_type)]
    return file_list


def calc_dice(img_result, gt):
    test_od = img_result[1]
    test_cup = img_result[2]
    od_gt = gt[1, :, :] // 255
    cup_gt = gt[2, :, :] // 255
    od_difference = np.where(test_od != od_gt, 1, 0)
    cup_difference = np.where(test_cup != cup_gt, 1, 0)
    od_dice = 1 - (np.sum(od_difference)) / (np.sum(test_od) + np.sum(od_gt))
    cup_dice = 1 - (np.sum(cup_difference)) / (np.sum(test_cup) + np.sum(cup_gt))
    return od_dice, cup_dice

<h3><center>Training Dataset Functions</center></h3>

In [7]:
def return_list(data_path, data_type):
    file_list = [file for file in os.listdir(data_path) if file.lower().endswith(data_type)]
    return file_list


class trainDataset(Dataset):
    def __init__(self, imgs_path, gts_path, crop_size, input_size, data_augmentation=False, data_polar=False):
        self.imgs_path = imgs_path
        self.gts_path = gts_path
        self.crop_size = crop_size
        self.input_size = input_size
        self.data_augmentation = data_augmentation
        self.data_polar = data_polar
        self.imgs, self.gts = self.get_dataset()
        self.p = 0.5

    def __len__(self):
        return self.imgs.shape[0]

    def disc_crop(self, org_img, crop_size, C_x, C_y):
        tmp_size = int(crop_size / 2)
        disc_region = np.zeros((crop_size, crop_size, org_img.shape[2]), dtype=org_img.dtype)
        gt_point = np.array([C_x - tmp_size, C_x + tmp_size, C_y - tmp_size, C_y + tmp_size], dtype=int)
        crop_point = [0, crop_size, 0, crop_size]

        if gt_point[0] < 0:
            crop_point[0] = abs(gt_point[0])
            gt_point[0] = 0

        if gt_point[2] < 0:
            crop_point[2] = abs(gt_point[2])
            gt_point[2] = 0

        if gt_point[1] > org_img.shape[0]:
            crop_point[1] = crop_point[1] - (gt_point[1] - org_img.shape[0])
            gt_point[1] = org_img.shape[0]

        if gt_point[3] > org_img.shape[1]:
            crop_point[3] = crop_point[3] - (gt_point[3] - org_img.shape[1])
            gt_point[3] = org_img.shape[1]

        disc_region[crop_point[0]:crop_point[1], crop_point[2]:crop_point[3], :] = org_img[gt_point[0]:gt_point[1], gt_point[2]:gt_point[3], :]

        return disc_region, crop_point, gt_point

    def get_dataset(self):
        file_imgs_list = return_list(self.imgs_path, '.png')

        imgs = []
        gts = []
        for file_img in file_imgs_list:
            image = cv2.imread(os.path.join(self.imgs_path, file_img[:-4] + '.png'))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            gt = np.zeros(np.shape(image), dtype=np.uint8)
            org_OD = cv2.imread(os.path.join(self.gts_path, file_img[:-4], 'SoftMap', file_img[:-4] + '_ODsegSoftmap.png'))
            org_OD = cv2.cvtColor(org_OD, cv2.COLOR_BGR2RGB)[:, :, 0]
            org_cup = cv2.imread(os.path.join(self.gts_path, file_img[:-4], 'SoftMap', file_img[:-4] + '_cupsegSoftmap.png'))
            org_cup = cv2.cvtColor(org_cup, cv2.COLOR_BGR2RGB)[:, :, 0]
            gt[org_OD > np.mean(np.unique(org_OD)), 1] = 255
            gt[org_cup > np.mean(np.unique(org_cup)), 2] = 255

            with open(os.path.join(self.gts_path, file_img[:-4], 'AvgBoundary', file_img[:-4] + '_diskCenter.txt')) as f:
                disk_center = np.array([int(i) for i in f.read().split(' ')])

            for crop_size in self.crop_size:
                crop_img, crop_point, gt_point = self.disc_crop(image, crop_size, disk_center[0], disk_center[1])
                crop_gt, _, _ = self.disc_crop(gt, crop_size, disk_center[0], disk_center[1])

                if self.data_polar is True:
                    crop_img = rotate(cv2.linearPolar(crop_img, (crop_size / 2, crop_size / 2), crop_size / 2,
                                                      cv2.INTER_NEAREST + cv2.WARP_FILL_OUTLIERS), -90)
                    crop_gt = rotate(cv2.linearPolar(crop_gt, (crop_size / 2, crop_size / 2), crop_size / 2,
                                                     cv2.INTER_NEAREST + cv2.WARP_FILL_OUTLIERS), -90)
                    crop_img = (crop_img * 255).astype(np.uint8)
                    crop_gt = (crop_gt * 255).astype(np.uint8)

                imgs.append((resize(crop_img, (self.input_size, self.input_size, 3), mode='constant') * 255).astype(np.uint8))
                gts.append((resize(crop_gt, (self.input_size, self.input_size, 3), mode='constant') * 255).astype(np.uint8))

        imgs = np.asarray(imgs)
        gts = np.asarray(gts)
        print('Train imgs shape: ' + str(imgs.shape))
        print('Train gts shape: ' + str(gts.shape))
        return imgs, gts

    @staticmethod
    def get_params(img, output_size):
        """Get parameters for ``crop`` for a random crop.
        Args:
            img (PIL Image): Image to be cropped.
            output_size (tuple): Expected output size of the crop.
        Returns:
            tuple: params (i, j, h, w) to be passed to ``crop`` for random crop.
        """
        w, h = img.size
        th, tw = output_size
        if w == tw and h == th:
            return 0, 0, h, w

        i = random.randint(0, h - th)
        j = random.randint(0, w - tw)
        return i, j, th, tw

    def __getitem__(self, idx):
        img = self.imgs[idx]
        gt = self.gts[idx]
        h, w = img.shape[0], img.shape[1]
        img = Image.fromarray(img)
        gt = Image.fromarray(gt)

        if self.data_augmentation is True and random.random() < 0.6:
            if random.random() > self.p:
                img = img.transpose(Image.FLIP_LEFT_RIGHT)
                gt = gt.transpose(Image.FLIP_LEFT_RIGHT)
            if random.random() > self.p:
                img = img.transpose(Image.FLIP_TOP_BOTTOM)
                gt = gt.transpose(Image.FLIP_TOP_BOTTOM)
            if random.random() > self.p:
                img = img.transpose(Image.ROTATE_90)
                gt = gt.transpose(Image.ROTATE_90)
            if random.random() > self.p:
                img = img.transpose(Image.ROTATE_180)
                gt = gt.transpose(Image.ROTATE_180)
            if random.random() > self.p:
                img = img.transpose(Image.ROTATE_270)
                gt = gt.transpose(Image.ROTATE_270)
            img = TF.pad(img, int(math.ceil(h * 1 / 8)))
            gt = TF.pad(gt, int(math.ceil(h * 1 / 8)))
            i, j, h, w = self.get_params(img, (h, w))
            img = img.crop((j, i, j + w, i + h))
            gt = gt.crop((j, i, j + w, i + h))

        return TF.to_tensor(img), TF.to_tensor(np.array(gt))


<h3><center>Test Dataset Functions</center></h3>

In [8]:
def return_list(data_path, data_type):
    file_list = [file for file in os.listdir(data_path) if file.lower().endswith(data_type)]
    return file_list


class testDataset(Dataset):
    def __init__(self, imgs_path, gts_path, crop_size, input_size, data_polar=False):
        self.imgs_path = imgs_path
        self.gts_path = gts_path
        self.crop_size = crop_size
        self.input_size = input_size
        self.data_polar = data_polar
        self.img_h = 0
        self.img_w = 0
        self.imgs, self.gts, self.disk_centers, self.crop_points, self.gt_points = self.get_dataset()
        self.p = 0.5

    def __len__(self):
        return self.imgs.shape[0]

    def disc_crop(self, org_img, crop_size, C_x, C_y):
        tmp_size = int(crop_size / 2)
        disc_region = np.zeros((crop_size, crop_size, org_img.shape[2]), dtype=org_img.dtype)  # (400, 400, 3)
        gt_point = np.array([C_x - tmp_size, C_x + tmp_size, C_y - tmp_size, C_y + tmp_size], dtype=int)  # [787, 1187, 146, 546]
        crop_point = [0, crop_size, 0, crop_size]  # [0, 400, 0, 400]

        if gt_point[0] < 0:
            crop_point[0] = abs(gt_point[0])
            gt_point[0] = 0

        if gt_point[2] < 0:
            crop_point[2] = abs(gt_point[2])
            gt_point[2] = 0

        if gt_point[1] > org_img.shape[0]:
            crop_point[1] = crop_point[1] - (gt_point[1] - org_img.shape[0])
            gt_point[1] = org_img.shape[0]

        if gt_point[3] > org_img.shape[1]:
            crop_point[3] = crop_point[3] - (gt_point[3] - org_img.shape[1])
            gt_point[3] = org_img.shape[1]

        disc_region[crop_point[0]:crop_point[1], crop_point[2]:crop_point[3], :] = org_img[gt_point[0]:gt_point[1], gt_point[2]:gt_point[3], :]

        return disc_region, crop_point, gt_point

    def get_dataset(self):
        file_imgs_list = return_list(self.imgs_path, '.png')

        imgs = []
        gts = []
        disk_centers = []
        crop_points = []
        gt_points = []
        for file_img in file_imgs_list:
            image = cv2.imread(os.path.join(self.imgs_path, file_img[:-4] + '.png'))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # print(image.shape)
            if self.img_h == 0 or self.img_w == 0:
                self.img_h, self.img_w = image.shape[0], image.shape[1]

            gt = np.zeros(np.shape(image), dtype=np.uint8)
            org_OD = cv2.imread(os.path.join(self.gts_path, file_img[:-4], 'SoftMap', file_img[:-4] + '_ODsegSoftmap.png'))
            org_OD = cv2.cvtColor(org_OD, cv2.COLOR_BGR2RGB)[:, :, 0]
            org_cup = cv2.imread(os.path.join(self.gts_path, file_img[:-4], 'SoftMap', file_img[:-4] + '_cupsegSoftmap.png'))
            org_cup = cv2.cvtColor(org_cup, cv2.COLOR_BGR2RGB)[:, :, 0]
            gt[org_OD > np.mean(np.unique(org_OD)), 1] = 255
            gt[org_cup > np.mean(np.unique(org_cup)), 2] = 255
            # print(gt.shape)
            gts.append(gt.astype(np.uint8))

            with open(os.path.join(self.gts_path, file_img[:-4], 'AvgBoundary', file_img[:-4] + '_diskCenter.txt')) as f:
                disk_centers.append(np.array([int(i) for i in f.read().split(' ')]))

            for crop_size in self.crop_size:
                crop_img, crop_point, gt_point = self.disc_crop(image, crop_size, disk_centers[-1][0], disk_centers[-1][1])

                if self.data_polar is True:
                    crop_img = rotate(cv2.linearPolar(crop_img, (crop_size / 2, crop_size / 2), crop_size / 2,
                                                      cv2.INTER_NEAREST + cv2.WARP_FILL_OUTLIERS), -90)
                    crop_img = (crop_img * 255).astype(np.uint8)

                imgs.append((resize(crop_img, (self.input_size, self.input_size, 3), mode='constant') * 255).astype(np.uint8))
                crop_points.append(crop_point)
                gt_points.append(gt_point)

        imgs = np.asarray(imgs)
        gts = np.asarray(gts)
        # print('Test imgs shape: ' + str(imgs.shape))
        # print('Test gts shape: ' + str(gts.shape) + '
        return imgs, gts, np.asarray(disk_centers), np.asarray(crop_points), np.asarray(gt_points)

    def __getitem__(self, idx):
        img = self.imgs[idx]
        gt = self.gts[idx]
        disk_center = self.disk_centers[idx]
        crop_point = self.crop_points[idx]
        gt_point = self.gt_points[idx]
        img = Image.fromarray(img)
        gt = Image.fromarray(gt)

        img = TF.to_tensor(img)
        gt = np.transpose(np.array(gt), (2, 0, 1))
        return img, gt, disk_center, crop_point, gt_point

<h3><center>Training Functions</center></h3>

In [10]:
torch.cuda.memory_summary(device=None, abbreviated=False)


model_names = ["CNN","RFCN"]

parser = argparse.ArgumentParser(description='CNN')
parser.add_argument('--device', type=str, default='0', help='GPU device (default: 0)')
parser.add_argument('--dataset', default='Drishti', choices=['Drishti'])
parser.add_argument('--data_path', type=str, default='datasets/', help='data path')

parser.add_argument('--model', type=str, default='RFCN', choices=model_names,
                    help='model architecture: ' + ' | '.join(model_names) + ' (default: RFCN)')
parser.add_argument('--recurrent_n', type=int, default=2, help='recurrent num (default: 2)')
parser.add_argument('--layers', type=int, default=4, help='net layers (default: 4)')

parser.add_argument('--loss_weights', type=str, default='0.1,0.1,0.1,0.1,0.6', help='loss_weights')
parser.add_argument('--optimizer', default='SGD', choices=['SGD', 'Adam'])
parser.add_argument('--batch_size', type=int, default=4, help='input batch size for training (default: 128)')
parser.add_argument('--epochs', type=int, default=400, help='number of epochs to train (default: 400)')

parser.add_argument('--lr', type=float, default=0.0001, help='learning rate')

parser.add_argument('--data_augmentation', action='store_true', default=False, help='data augmentation')
parser.add_argument('--data_polar', action='store_true', default=False, help='data polar')
parser.add_argument('--input_size', type=int, default=400, help='input imgs size (default: 400)')
parser.add_argument('--train_crop_size', type=str, default='400,500,550,600,650,700,750,800,850,900', help='input image size')
parser.add_argument('--test_crop_size', type=int, default=700, help='test image size (default: 700)')

parser.add_argument('--threshold_confusion', default=0.4, type=float, help='threshold_confusion')
parser.add_argument('--seed', type=int, default=1234, help='random seed (default: 1234)')

# use last save model
parser.add_argument('--load_last', action='store_true', default=False, help='load last model')
parser.add_argument('--load_path', type=str, default='logs/', help='load model path')
parser.add_argument('--logs_path', type=str, default='logs/', help='load model path')


parser.add_argument('-f')
args = vars(parser.parse_args())
os.environ['CUDA_VISIBLE_DEVICES'] = args['device']
cudnn.benchmark = True
torch.cuda.is_available()
torch.manual_seed(args['seed'])
torch.cuda.manual_seed(args['seed'])

threshold_confusion = args['threshold_confusion']

if str(args['logs_path']).endswith('/') is False:
    args['logs_path'] += '/'

if args['load_path'] is not None and str(args['load_path']).endswith('/') is False:
    args['load_path'] += '/'

if args['load_last'] is False:
    mkdir_p(args['logs_path'] + args['dataset'] + '/' + args['model'] + '/')
    index = np.sort(np.array(os.listdir(args['logs_path'] + args['dataset'] + '/' + args['model'] + '/'), dtype=int))
    index = index.max() + 1 if len(index) > 0 else 1
    basic_path = args['logs_path'] + args['dataset'] + '/' + args['model'] + '/' + str(index) + '/'
    mkdir_p(basic_path)
    args['load_path'] = basic_path
    max_OD_Dice, max_OD_Dice_cup_Dice, max_cup_Dice, max_cup_Dice_OD_Dice = 0., 0., 0., 0.
    cur_epoch = 0
    logs = [['epoch', 'OD_Dice', 'max_OD_Dice', 'max_OD_Dice_cup_Dice', 'cup_Dice', 'max_cup_Dice', 'max_cup_Dice_OD_Dice']]
else:
    basic_path = args['load_path']
    assert os.path.exists(basic_path), 'Error: Folder not exists'
    assert os.path.isfile(basic_path + 'checkpoints/last.pt'), 'Error: no checkpoint file found!'
    checkpoint = torch.load(basic_path + 'checkpoints/last.pt')
    checkpoint['args']['load_last'] = args['load_last']
    checkpoint['args']['load_path'] = args['load_path']
    args = checkpoint['args']
    max_OD_Dice = checkpoint['max_OD_Dice']
    max_OD_Dice_cup_Dice = checkpoint['max_OD_Dice_cup_Dice']
    max_cup_Dice = checkpoint['max_cup_Dice']
    max_cup_Dice_OD_Dice = checkpoint['max_cup_Dice_OD_Dice']
    cur_epoch = checkpoint['epoch'] + 1
    logs = checkpoint['logs']
    assert cur_epoch < args['epochs'], 'Done，cur_epoch: {}，epochs: {}'.format(cur_epoch, args['epochs'])

print('Current Folder： ' + basic_path)
mkdir_p(basic_path + 'checkpoints/periods/')
mkdir_p(basic_path + 'tensorboard/')
with open(basic_path + 'args.txt', 'w+') as f:
    for arg in args:
        print(str(arg) + ': ' + str(args[arg]))
        f.write(str(arg) + ': ' + str(args[arg]) + '\n')


net = RFCN(num_classes=3, layers=args['layers'], filters=32, inplanes=3, recurrent_n=args['recurrent_n'])

criterions = []
for i in range(args['layers'] + 1):
    criterions.append(nn.CrossEntropyLoss().cuda())

loss_weights = [float(loss_weight) for loss_weight in args['loss_weights'].split(',')]

net.cuda()
if args['optimizer'] == 'Adam':
    optimizer = optim.Adam(net.parameters(), lr=args['lr'])
elif args['optimizer'] == 'SGD':
    optimizer = optim.SGD(net.parameters(), lr=args['lr'], momentum=0.9, weight_decay=5e-4)

if args['load_last'] is True and cur_epoch > 0:
    net.load_state_dict(checkpoint['net'], strict=False)
    print('load path: ' + basic_path + 'checkpoints/last.pt')

if args['dataset'] == 'Drishti':
    train_set = trainDataset(imgs_path='/content/drive/MyDrive/Datasets/Drishti-GS1_files/Training/Images',
                             gts_path='/content/drive/MyDrive/Datasets/Drishti-GS1_files/Training/GT',
                             crop_size=[int(train_crop_size) for train_crop_size in args['train_crop_size'].split(',')],
                             input_size=args['input_size'],
                             data_augmentation=args['data_augmentation'],
                             data_polar=args['data_polar'])
    test_set = testDataset(imgs_path='/content/drive/MyDrive/Datasets/Drishti-GS1_files/Test/Images',
                           gts_path='/content/drive/MyDrive/Datasets/Drishti-GS1_files/Test/Test_GT',
                           crop_size=[args['test_crop_size']],
                           input_size=args['input_size'],
                           data_polar=args['data_polar'])
    

train_loader = DataLoader(train_set, batch_size=4, shuffle=True, num_workers=0)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=0)

ts_writer = SummaryWriter(log_dir=basic_path + 'tensorboard/', comment=args['model'])
args_str = ''
for arg in args:
    args_str += str(arg) + ': ' + str(args[arg]) + '<br />'
ts_writer.add_text('args', args_str, cur_epoch)


def train():
    global max_OD_Dice, max_OD_Dice_cup_Dice, max_cup_Dice, max_cup_Dice_OD_Dice
    for epoch in range(cur_epoch, args['epochs']):
        # train network
        net.train()
        train_loss = 0
        progress_bar = tqdm(train_loader)
        for batch_idx, (inputs, targets) in enumerate(progress_bar):
            progress_bar.set_description('Epoch {}/{}'.format(epoch + 1, args['epochs']))

            inputs = Variable(inputs.cuda().detach())
            targets = Variable(targets.long().cuda().detach())

            optimizer.zero_grad()
            outputs = net(inputs)

            loss_ = []
            for criterion, output, loss_weight in zip(criterions, outputs, loss_weights):
                loss_od = criterion(output[:, (0, 1), :, :], targets[:, 1, :, :].reshape(targets.shape[0], targets.shape[2], targets.shape[3]))
                loss_cup = criterion(output[:, (0, 2), :, :], targets[:, 2, :, :].reshape(targets.shape[0], targets.shape[2], targets.shape[3]))
                loss_.append(loss_weight * (0.5 * loss_od + 0.5 * loss_cup))

            loss = sum(loss_)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            progress_bar.set_postfix(loss='%.3f' % (train_loss / (batch_idx + 1)))

        net.eval()
        with torch.no_grad():
            OD_Dices = []
            cup_Dices = []
            for batch_idx, (test_input, test_target, test_disk_center, test_crop_point, test_gt_point) in enumerate(test_loader):
                test_input = test_input.detach().cuda()
                test_outputs = net(test_input)
                disc_predict = torch.nn.functional.softmax(test_outputs[-1][:, (0, 1), :, :], dim=1).cpu().data.numpy()[0]
                disc_map = (resize(disc_predict[1, :, :], (args['test_crop_size'], args['test_crop_size']), mode='constant') * 255).astype(np.uint8)
                cup_predict = torch.nn.functional.softmax(test_outputs[-1][:, (0, 2), :, :], dim=1).cpu().data.numpy()[0]
                cup_map = (resize(cup_predict[1, :, :], (args['test_crop_size'], args['test_crop_size']), mode='constant') * 255).astype(np.uint8)
                if args['data_polar'] is True:
                    disc_map[-round(args['test_crop_size'] / 3):, :] = 0
                    cup_map[-round(args['test_crop_size'] / 2):, :] = 0
                    disc_map = cv2.linearPolar(rotate(disc_map, 90), (args['test_crop_size'] / 2, args['test_crop_size'] / 2),
                                               args['test_crop_size'] / 2, cv2.WARP_FILL_OUTLIERS + cv2.WARP_INVERSE_MAP)
                    cup_map = cv2.linearPolar(rotate(cup_map, 90), (args['test_crop_size'] / 2, args['test_crop_size'] / 2),
                                              args['test_crop_size'] / 2, cv2.WARP_FILL_OUTLIERS + cv2.WARP_INVERSE_MAP)

                disc_map = np.array(BW_img(disc_map, threshold_confusion), dtype=int)
                cup_map = np.array(BW_img(cup_map, threshold_confusion), dtype=int)
                test_target = test_target[0].data.numpy()
                img_result = np.zeros(test_target.shape, dtype=np.int8)
                img_result[1, test_gt_point[0][0]:test_gt_point[0][1], test_gt_point[0][2]:test_gt_point[0][3]] = \
                    disc_map[test_crop_point[0][0]:test_crop_point[0][1], test_crop_point[0][2]:test_crop_point[0][3]]
                img_result[2, test_gt_point[0][0]:test_gt_point[0][1], test_gt_point[0][2]:test_gt_point[0][3]] = \
                    cup_map[test_crop_point[0][0]:test_crop_point[0][1], test_crop_point[0][2]:test_crop_point[0][3]]

                OD_Dice, cup_Dice = calc_dice(img_result, test_target)
                OD_Dices.append(OD_Dice)
                cup_Dices.append(cup_Dice)

            OD_Dice = np.mean(OD_Dices)
            cup_Dice = np.mean(cup_Dices)

            if max_OD_Dice < OD_Dice:
                max_OD_Dice = OD_Dice
                max_OD_Dice_cup_Dice = cup_Dice
            if max_cup_Dice < cup_Dice:
                max_cup_Dice_OD_Dice = OD_Dice
                max_cup_Dice = cup_Dice

            logs.append([epoch, OD_Dice, max_OD_Dice, max_OD_Dice_cup_Dice, cup_Dice, max_cup_Dice, max_cup_Dice_OD_Dice])
            state = {
                'net': net.state_dict(),
                'max_OD_Dice': max_OD_Dice,
                'max_OD_Dice_cup_Dice': max_OD_Dice_cup_Dice,
                'max_cup_Dice': max_cup_Dice,
                'max_cup_Dice_OD_Dice': max_cup_Dice_OD_Dice,
                'epoch': epoch,
                'logs': logs,
                'args': args
            }

            torch.save(state, basic_path + 'checkpoints/periods/{}.pt'.format(epoch))
            torch.save(state, basic_path + 'checkpoints/last.pt')

            lr = get_lr(optimizer)
            ts_writer.add_scalar('train/loss', train_loss / len(train_loader), epoch)
            ts_writer.add_scalar('train/lr', lr, epoch)
            ts_writer.add_scalar('test/OD_Dice', OD_Dice, epoch)
            ts_writer.add_scalar('test/cup_Dice', cup_Dice, epoch)
            tqdm.write('     OD_Dice: {:.4f},     cup_Dice: {:.4f}'.format(OD_Dice, cup_Dice))
            tqdm.write(' max_OD_Dice: {:.4f},     cup_Dice: {:.4f}'.format(max_OD_Dice, max_OD_Dice_cup_Dice))
            tqdm.write('     OD_Dice: {:.4f}, max_cup_Dice: {:.4f}'.format(max_cup_Dice_OD_Dice, max_cup_Dice))
            tqdm.write('learning rate: {:.4f}'.format(lr))


if __name__ == '__main__':
    train()
    ts_writer.close()

Current Folder： logs/Drishti/RFCN/16/
device: 0
dataset: Drishti
data_path: datasets/
model: RFCN
recurrent_n: 2
layers: 4
loss_weights: 0.1,0.1,0.1,0.1,0.6
optimizer: SGD
batch_size: 4
epochs: 400
lr: 0.0001
data_augmentation: False
data_polar: False
input_size: 400
train_crop_size: 400,500,550,600,650,700,750,800,850,900
test_crop_size: 700
threshold_confusion: 0.4
seed: 1234
load_last: False
load_path: logs/Drishti/RFCN/16/
logs_path: logs/
f: /root/.local/share/jupyter/runtime/kernel-3ac45318-c9e3-40ff-aa15-8fb2a5491699.json
Train imgs shape: (500, 400, 400, 3)
Train gts shape: (500, 400, 400, 3)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
Epoch 1/400: 100%|██████████| 125/125 [01:30<00:00,  1.38it/s, loss=0.524]


     OD_Dice: 0.3805,     cup_Dice: 0.2292
 max_OD_Dice: 0.3805,     cup_Dice: 0.2292
     OD_Dice: 0.3805, max_cup_Dice: 0.2292
learning rate: 0.0001


Epoch 2/400: 100%|██████████| 125/125 [01:27<00:00,  1.43it/s, loss=0.370]


     OD_Dice: 0.3805,     cup_Dice: 0.2292
 max_OD_Dice: 0.3805,     cup_Dice: 0.2292
     OD_Dice: 0.3805, max_cup_Dice: 0.2292
learning rate: 0.0001


...

     OD_Dice: 0.9315,     cup_Dice: 0.8598
 max_OD_Dice: 0.9399,     cup_Dice: 0.8637
     OD_Dice: 0.9399, max_cup_Dice: 0.8637
learning rate: 0.0001


Epoch 399/400: 100%|██████████| 125/125 [01:28<00:00,  1.42it/s, loss=0.011]


     OD_Dice: 0.9354,     cup_Dice: 0.8608
 max_OD_Dice: 0.9399,     cup_Dice: 0.8637
     OD_Dice: 0.9399, max_cup_Dice: 0.8637
learning rate: 0.0001


Epoch 400/400: 100%|██████████| 125/125 [01:28<00:00,  1.42it/s, loss=0.011]


     OD_Dice: 0.9362,     cup_Dice: 0.8614
 max_OD_Dice: 0.9399,     cup_Dice: 0.8637
     OD_Dice: 0.9399, max_cup_Dice: 0.8637
learning rate: 0.0001


In [9]:
model_names = ["CNN","RFCN"]

parser = argparse.ArgumentParser(description='CNN')
# use last save model
parser.add_argument('--device', type=str, default='0', help='GPU device (default: 0)')
parser.add_argument('--test_crop_size', type=str, default='700,750,800,850,900', help='input image size')
parser.add_argument('--threshold_confusion', default=0.4, type=float, help='threshold_confusion')
parser.add_argument('--check_path', type=str, default='/content/drive/MyDrive/Datasets/Drishti-GS1_files/LOG/399.pt', help='load model path')

parser.add_argument('-f')
args = vars(parser.parse_args())

checkpoint = torch.load(args['check_path'])
checkpoint['args']['device'] = args['device']
checkpoint['args']['check_path'] = args['check_path']
checkpoint['args']['test_crop_size'] = args['test_crop_size']
checkpoint['args']['threshold_confusion'] = args['threshold_confusion']
args = checkpoint['args']
logs = checkpoint['logs']
print(logs[-1])

os.environ['CUDA_VISIBLE_DEVICES'] = args['device']
cudnn.benchmark = True
torch.cuda.is_available()
torch.manual_seed(args['seed'])
torch.cuda.manual_seed(args['seed'])
threshold_confusion = args['threshold_confusion']

path_arr = args['check_path'].split('/')
data_save_path = mkdir_p(args['check_path'][:-3] + '_result/')

disc_list = [int(test_crop_size) for test_crop_size in args['test_crop_size'].split(',')]


net = RFCN(num_classes=3, layers=args['layers'], filters=32, inplanes=3)
net.eval().cuda()
net.load_state_dict(checkpoint['net'], strict=False)

file = open(str(data_save_path) + 'performance.txt', 'w+')
for disc_idx in range(len(disc_list)):
    crop_size = disc_list[disc_idx]
    locals()['OD_Dice_' + str(crop_size)] = []
    locals()['cup_Dice_' + str(crop_size)] = []
    test_set = testDataset(imgs_path='/content/drive/MyDrive/Datasets/Drishti-GS1_files/Test/Images/',
                           gts_path='/content/drive/MyDrive/Datasets/Drishti-GS1_files/Test/Test_GT/',
                           crop_size=[crop_size],
                           input_size=args['input_size'],
                           data_polar=args['data_polar'])

    test_loader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=0)

    with torch.no_grad():
        OD_Dices = []
        cup_Dices = []
        for batch_idx, (test_input, test_target, test_disk_center, test_crop_point, test_gt_point) in enumerate(test_loader):
            test_input = test_input.detach().cuda()
            test_outputs = net(test_input)
            disc_predict = torch.nn.functional.softmax(test_outputs[-1][:, (0, 1), :, :], dim=1).cpu().data.numpy()[0]
            disc_map = (resize(disc_predict[1, :, :], (crop_size, crop_size), mode='constant') * 255).astype(np.uint8)
            cup_predict = torch.nn.functional.softmax(test_outputs[-1][:, (0, 2), :, :], dim=1).cpu().data.numpy()[0]
            cup_map = (resize(cup_predict[1, :, :], (crop_size, crop_size), mode='constant') * 255).astype(np.uint8)
            if args['data_polar'] is True:
                disc_map[-round(crop_size / 3):, :] = 0
                cup_map[-round(crop_size / 2):, :] = 0
                disc_map = cv2.linearPolar(rotate(disc_map, 90), (crop_size / 2, crop_size / 2),
                                           crop_size / 2, cv2.WARP_FILL_OUTLIERS + cv2.WARP_INVERSE_MAP)
                cup_map = cv2.linearPolar(rotate(cup_map, 90), (crop_size / 2, crop_size / 2),
                                          crop_size / 2, cv2.WARP_FILL_OUTLIERS + cv2.WARP_INVERSE_MAP)

            disc_map = np.array(BW_img(disc_map, threshold_confusion), dtype=int)
            cup_map = np.array(BW_img(cup_map, threshold_confusion), dtype=int)
            test_target = test_target[0].data.numpy()
            img_result = np.zeros(test_target.shape, dtype=np.int8)
            img_result[1, test_gt_point[0][0]:test_gt_point[0][1], test_gt_point[0][2]:test_gt_point[0][3]] = \
                disc_map[test_crop_point[0][0]:test_crop_point[0][1], test_crop_point[0][2]:test_crop_point[0][3]]
            img_result[2, test_gt_point[0][0]:test_gt_point[0][1], test_gt_point[0][2]:test_gt_point[0][3]] = \
                cup_map[test_crop_point[0][0]:test_crop_point[0][1], test_crop_point[0][2]:test_crop_point[0][3]]

            OD_Dice, cup_Dice = calc_dice(img_result, test_target)
            eval('OD_Dice_' + str(crop_size)).append(OD_Dice)
            eval('cup_Dice_' + str(crop_size)).append(cup_Dice)

        print("%3s mean OD_Dice: %4f + %4f, mean cup_Dice: %4f + %4f" % (crop_size,
                                                                         np.mean(eval('OD_Dice_' + str(crop_size))),
                                                                         np.std(eval('OD_Dice_' + str(crop_size))),
                                                                         np.mean(eval('cup_Dice_' + str(crop_size))),
                                                                         np.std(eval('cup_Dice_' + str(crop_size)))))
        file.write("%3s,%4f,%4f,%4f,%4f\n" % (crop_size,
                                                      np.mean(eval('OD_Dice_' + str(crop_size))),
                                                      np.std(eval('OD_Dice_' + str(crop_size))),
                                                      np.mean(eval('cup_Dice_' + str(crop_size))),
                                                      np.std(eval('cup_Dice_' + str(crop_size)))))

[399, 0.9362217612288978, 0.9399288078286311, 0.8636579410654747, 0.8613787591102227, 0.8636579410654747, 0.9399288078286311]


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


700 mean OD_Dice: 0.936222 + 0.070652, mean cup_Dice: 0.861379 + 0.123767
750 mean OD_Dice: 0.937347 + 0.075064, mean cup_Dice: 0.861878 + 0.119726
800 mean OD_Dice: 0.903219 + 0.157381, mean cup_Dice: 0.790822 + 0.212251
850 mean OD_Dice: 0.700915 + 0.371627, mean cup_Dice: 0.611540 + 0.356638
900 mean OD_Dice: 0.501315 + 0.428304, mean cup_Dice: 0.451702 + 0.397977
